1. По критерию Пирсона при уровне значимости α = 0,05 проверить
нулевую гипотезу о нормальном распределении:
    - случайной величины «возраст». Для этого разбить массив исходных
данных на 7 групп с равными интервалами (концы интервала –
целые).
    - случайной величины «средний возраст», используя выборку
средних, полученную в лабораторной работе №2.

In [1]:
f = open("moscow_2021.txt", "r")
data = []
for num in f:
    num = num.strip()
    if num:
        data.append(int(num))


In [2]:
import math
import scipy.stats as stat

interval_count = 7
min_age = min(data)
max_age = max(data)

# нахождение границ интервалов
interaval_range = max_age - min_age 
interval_length = math.ceil(interaval_range / interval_count)

intervals = []
for i in range(interval_count + 1):
    interval_boundary = min_age + i * interval_length
    intervals.append(math.ceil(interval_boundary))
print(intervals)


[14, 23, 32, 41, 50, 59, 68, 77]


In [3]:
import numpy as np
from scipy.stats import chisquare

def test_normal_distribution(data):
    # Ваши данные о возрасте
    ages = np.array(data)
    

    # Разбить данные на 7 групп с равными интервалами
    bins = np.linspace(min(ages), max(ages), 8)
    hist, bin_edges = np.histogram(ages, bins=bins)

    # Рассчитать теоретические частоты для нормального распределения
    mu = np.mean(ages)
    sigma = np.std(ages)
    expected_freq = []
    for i in range(len(bin_edges)-1):
        start = bin_edges[i]
        end = bin_edges[i+1]
        prob = (1 / (sigma * np.sqrt(2 * np.pi))) * np.exp(-((start + end) / 2 - mu)**2 / (2 * sigma**2)) * (end - start)
        expected_freq.append(prob * len(ages))
    expected_freq = np.array(expected_freq)

    # Масштабируем ожидаемые частоты, чтобы их сумма была равна сумме наблюдаемых частот
    expected_freq = expected_freq * np.sum(hist) / np.sum(expected_freq)

    # Проверка критерием Пирсона
    chi2_stat, p_value = chisquare(hist, expected_freq)
    df = len(hist) - 1  # степени свободы
    alpha = 0.05

    print("Среднее:", mu)
    print("Стандартное отклонение:", sigma)

    print("Хи-квадрат:", chi2_stat)
    print("p-значение:", p_value)
    print("Степени свободы:", df)

    if p_value < alpha:
        print("Отвергаем нулевую гипотезу")
    else:
        print("Не отвергаем нулевую гипотезу")

# Call the function with your data
test_normal_distribution(data)


Среднее: 35.37300681614903
Стандартное отклонение: 12.038146371631095
Хи-квадрат: 2903.5525084801898
p-значение: 0.0
Степени свободы: 6
Отвергаем нулевую гипотезу


Получаем выборку из прошлого проекта


In [4]:
from collections import Counter
import math
import random

def average(data):
    average_sum = 0
    for num in data:
        average_sum += num
    average_num = average_sum/len(data)
    return average_num

def variance(data):
    variance_sum = 0
    average_of_data = average(data)
    num_count = Counter(data)
    for num in data:
        
        variance_sum += ((num - average_of_data)**2)
    return ((variance_sum)/len(data))

#2Φ(𝑡) = 0,95 ⇒ Φ(𝑡) = 0,475
#Значению 0,475 в таблице функции Лапласа соответствует значение
#аргумента 𝑡 = 1,96
num_of_samples = 36
year = 3
student_criteria = 1.96
n = round((student_criteria * student_criteria * variance(data))/(year*year))
average_values = []
samples = []

for i in range(0,num_of_samples):
    temp_data = []
    for j in range(0,n):
        temp_data.append(data[random.randint(0,len(data)-1)])
    average_values.append(average(temp_data))
    samples.append(temp_data)
    temp_data = []

print(average_values)
test_normal_distribution(average_values)

[34.306451612903224, 34.306451612903224, 33.403225806451616, 36.46774193548387, 35.32258064516129, 36.45161290322581, 34.483870967741936, 36.91935483870968, 36.03225806451613, 38.53225806451613, 36.67741935483871, 39.16129032258065, 34.82258064516129, 33.87096774193548, 37.064516129032256, 36.225806451612904, 31.806451612903224, 35.88709677419355, 34.38709677419355, 39.38709677419355, 30.887096774193548, 35.854838709677416, 35.29032258064516, 33.903225806451616, 32.225806451612904, 36.53225806451613, 36.95161290322581, 35.79032258064516, 35.435483870967744, 35.596774193548384, 33.74193548387097, 36.91935483870968, 35.08064516129032, 35.32258064516129, 36.03225806451613, 35.33870967741935]
Среднее: 35.45609318996415
Стандартное отклонение: 1.796638778861766
Хи-квадрат: 9.442151839189734
p-значение: 0.15019596392594958
Степени свободы: 6
Не отвергаем нулевую гипотезу


### Проверка статистических гипотез

Проверить нулевую гипотезу о равенстве двух дисперсий генеральных совокупностей при уровне значимость 0.05

In [6]:
import scipy.stats as stats
s_1 = variance(samples[0])
s_2 = variance(samples[1])


if s_1 > s_2:
    f = s_1/s_2
else:
    f = s_2/s_1

alpha = 0.05

df1 = len(samples[0]) - 1
df2 = len(samples[1]) - 1


print(f)

1.1306732839948843


$𝐻1 :𝐷1 > 𝐷2$

In [7]:
f_teor = stats.f.ppf(1 - alpha, df1, df2)

print(f"Значение F теоритическое = {f_teor}")
if f < f_teor:
    print("Гипотеза принимается")
else:
    print("Гипотеза отвергается")


Значение F теоритическое = 1.5288331472305499
Гипотеза принимается


$𝐻1:𝐷1 ≠ 𝐷2$

In [8]:
f_teor = stats.f.ppf(1 - alpha/2, df1, df2)
print(f"Значение F теоритическое = {f_teor}")
if f < f_teor:
    print("Гипотеза принимается")
else:
    print("Гипотеза отвергается")

Значение F теоритическое = 1.6596651974088976
Гипотеза принимается
